In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# случайный лес

In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, classification_report
from imblearn.over_sampling import SMOTE

In [3]:
# Укажем путь к данным
DATA_PATH = r"/content/drive/MyDrive/home-credit-default-risk"

# Загружаем данные
app_train = pd.read_csv(os.path.join(DATA_PATH, "application_train.csv"))

# Выбираем только числовые признаки
num_features = app_train.select_dtypes(include=[np.number]).drop(columns=["TARGET", "SK_ID_CURR"])

# Заполняем пропущенные значения медианными
imputer = SimpleImputer(strategy='median')
num_features_imputed = pd.DataFrame(imputer.fit_transform(num_features), columns=num_features.columns)

# Создание новых признаков
num_features_imputed['CREDIT_INCOME_RATIO'] = num_features_imputed['AMT_CREDIT'] / (num_features_imputed['AMT_INCOME_TOTAL'] + 1)
num_features_imputed['ANNUITY_INCOME_RATIO'] = num_features_imputed['AMT_ANNUITY'] / (num_features_imputed['AMT_INCOME_TOTAL'] + 1)
num_features_imputed['DAYS_EMPLOYED_RATIO'] = num_features_imputed['DAYS_EMPLOYED'] / (num_features_imputed['DAYS_BIRTH'] + 1)

# Разделение на обучающую и тестовую выборки
X = num_features_imputed
y = app_train["TARGET"]

In [4]:
# Балансировка данных с SMOTE (увеличение выборки класса 1)
smote = SMOTE(sampling_strategy=0.3, random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, stratify=y_resampled, random_state=42)

In [5]:
from tqdm import tqdm

# Количество деревьев
n_estimators = 500

# Инициализация модели с warm_start=True для поэтапного добавления деревьев
rf_model = RandomForestClassifier(
    n_estimators=1,  # начнем с 1 дерева
    max_depth=10,
    min_samples_split=10,
    min_samples_leaf=4,
    class_weight='balanced',
    random_state=42,
    warm_start=True  # позволяет наращивать количество деревьев
)

# Обучение с прогресс-баром
for i in tqdm(range(1, n_estimators + 1), desc="Training Random Forest"):
    rf_model.set_params(n_estimators=i)
    rf_model.fit(X_train, y_train)


Training Random Forest:   0%|          | 0/500 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_forest.py:861: UserWarning: class_weight presets "balanced" or "balanced_subsample" are not recommended for warm_start if the fitted data differs from the full dataset. In order to use "balanced" weights, use compute_class_weight ("balanced", classes, y). In place of y you can use a large enough sample of the full training set target to properly estimate the class frequency distributions. Pass the resulting weights as the class_weight parameter.
  warn(
Training Random Forest:   0%|          | 1/500 [00:01<08:29,  1.02s/it]/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_forest.py:861: UserWarning: class_weight presets "balanced" or "balanced_subsample" are not recommended for warm_start if the fitted data differs from the full dataset. In order to use "balanced" weights, use compute_class_weight ("balanced", classes, y). In place of y you can use a large en

In [6]:
# Предсказания
y_pred = rf_model.predict(X_test)
y_prob = rf_model.predict_proba(X_test)[:, 1]

In [7]:
# Оценка качества модели
auc_score = roc_auc_score(y_test, y_prob)
print(f"AUC-ROC Score (Random Forest): {auc_score:.4f}")
print(classification_report(y_test, y_pred))

AUC-ROC Score (Random Forest): 0.9077
              precision    recall  f1-score   support

           0       0.92      0.94      0.93     56538
           1       0.78      0.74      0.76     16961

    accuracy                           0.89     73499
   macro avg       0.85      0.84      0.85     73499
weighted avg       0.89      0.89      0.89     73499

